In [1]:
import os

In [2]:
%pwd

'c:\\Users\\dimpa\\Downloads\\PROJECT - HEALTHCARE\\chest-cancer-classification-using-mlflow-dvc\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\dimpa\\Downloads\\PROJECT - HEALTHCARE\\chest-cancer-classification-using-mlflow-dvc'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list
    params_learning_rate: float

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

        

    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "CHEST-CT-SCAN-DATASET")
        create_directories([
            Path(training.root_dir)
        ])


        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE,
            params_learning_rate=params.LEARNING_RATE   # ✅ Add this line
        )

        return training_config

In [8]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [9]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path,
        compile=False
        )

        self.model.compile(
            optimizer=tf.keras.optimizers.Adam(
                learning_rate=self.config.params_learning_rate
            ),
            loss="categorical_crossentropy",
            metrics=["accuracy"]
        )    

    def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)



    
    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )



In [10]:
from cnnClassifier.config.configuration import ConfigurationManager

try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
    
except Exception as e:
    raise e

[2026-02-18 10:48:50,964: INFO: common: yaml file: config\config.yaml loaded successfully]
[2026-02-18 10:48:50,966: INFO: common: yaml file: params.yaml loaded successfully]
[2026-02-18 10:48:50,968: INFO: common: created directory at: artifacts]
[2026-02-18 10:48:50,969: INFO: common: created directory at: artifacts\training]
Found 68 images belonging to 2 classes.
Found 275 images belonging to 2 classes.
Epoch 1/25
17/17 ━━━━━━━━━━━━━━━━━━━━ 30s 2s/step - accuracy: 0.6525 - loss: 7.8469 - val_accuracy: 0.8750 - val_loss: 0.4016
Epoch 2/25
 1/17 ━━━━━━━━━━━━━━━━━━━━ 24s 2s/step - accuracy: 0.8750 - loss: 1.4907

d:\miniconda3\envs\ChestCancer\Lib\site-packages\keras\src\trainers\epoch_iterator.py:116: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


17/17 ━━━━━━━━━━━━━━━━━━━━ 8s 404ms/step - accuracy: 0.8750 - loss: 1.4907 - val_accuracy: 0.8750 - val_loss: 0.2706
Epoch 3/25
17/17 ━━━━━━━━━━━━━━━━━━━━ 32s 2s/step - accuracy: 0.8726 - loss: 1.4017 - val_accuracy: 0.9844 - val_loss: 0.0571
Epoch 4/25
17/17 ━━━━━━━━━━━━━━━━━━━━ 7s 375ms/step - accuracy: 0.9375 - loss: 0.9009 - val_accuracy: 0.8750 - val_loss: 0.3820
Epoch 5/25
17/17 ━━━━━━━━━━━━━━━━━━━━ 33s 2s/step - accuracy: 0.8726 - loss: 1.5353 - val_accuracy: 1.0000 - val_loss: 0.0029
Epoch 6/25
17/17 ━━━━━━━━━━━━━━━━━━━━ 8s 413ms/step - accuracy: 0.8125 - loss: 2.7860 - val_accuracy: 1.0000 - val_loss: 0.0095
Epoch 7/25
17/17 ━━━━━━━━━━━━━━━━━━━━ 35s 2s/step - accuracy: 0.9305 - loss: 0.6514 - val_accuracy: 1.0000 - val_loss: 7.1850e-04
Epoch 8/25
17/17 ━━━━━━━━━━━━━━━━━━━━ 9s 433ms/step - accuracy: 0.9375 - loss: 0.2372 - val_accuracy: 1.0000 - val_loss: 2.3838e-05
Epoch 9/25
17/17 ━━━━━━━━━━━━━━━━━━━━ 37s 2s/step - accuracy: 0.9522 - loss: 0.6358 - val_accuracy: 1.0000 - val_